In [19]:
import os
import librosa 
import torch
import matplotlib
import matplotlib.pyplot as plt
import random
import IPython.display as ipd

matplotlib.use('Agg')

%matplotlib inline

from matplotlib.colors import Normalize
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from argparse import ArgumentParser
from omegaconf import OmegaConf
from unicodedata import normalize
import re

#from utils.utils import get_commit_hash
from lightning_model import Wavegrad2
from text import Language
from text.symbols import eng_symbols
from g2p_en import G2p
from dataloader import TextAudioDataset


if __name__ == '__main__':
    parser = ArgumentParser()
    parser.add_argument('-c', '--config', type=str, required=True,
                        help="path of configuration yaml file")
    parser.add_argument('-g', '--gpus', type=str, default=None,
        help="Number of gpus to use (e.g. '0,1,2,3'). Will use all if not given.")
    parser.add_argument('-n', '--name', type=str, required=True,
                        help="Name of the run.")
    parser.add_argument('-p', '--checkpoint_path', type=str, default=None,
                        help="path of checkpoint for resuming")

    lu=['-c','hparameter.yaml','-n','trash','-p']
    
#     lu.append('/workspace/checkpoint/wavegrad2_07_02_08_epoch=649.ckpt')
    lu.append('/workspace/checkpoint/wavegrad2_07_08_07_epoch=87.ckpt')
    speaker = 'LJSpeech'
    
    args = parser.parse_args(lu)

    net=Wavegrad2.load_from_checkpoint(args.checkpoint_path).cuda().eval()
    net.freeze()
    
    hparams =net.hparams

In [29]:

#     text=input('문자열을 입력하세요: ')

    text="On Friday night in Bridgeport expect a temperature of minus four degrees Fahrenheit."
    
    g2p = G2p()
    phones = []
    words = re.split(r"([,;.\-\?\!\s+])", text)
    for w in words:
        phones += list(filter(lambda p: p != " ", g2p(w)))
    phones = "{" + "}{".join(phones) + "}"
    phones = re.sub(r"\{[^\w\s]?\}", "{sp}", phones)
    print('g2p: ',phones)
    
    trainset = TextAudioDataset(hparams, hparams.data.train_dir, hparams.data.train_meta, train=False)

    text=trainset.get_text(phones)
    text=text.unsqueeze(0)
    
    speaker_dict = {spk: idx for idx, spk in enumerate(hparams.data.speakers)}
    spk_id=[speaker_dict[speaker]]
    spk_id = torch.LongTensor(spk_id)
    
    text=text.cuda()
    spk_id=spk_id.cuda()
    
    wav_recon, align, dur, sigma = net.inference(text,spk_id, pace=1.0)


g2p:  {AA1}{N}{F}{R}{AY1}{D}{IY0}{N}{AY1}{T}{IH0}{N}{B}{R}{IH1}{JH}{P}{AO2}{R}{T}{IH0}{K}{S}{P}{EH1}{K}{T}{AH0}{T}{EH1}{M}{P}{R}{AH0}{CH}{ER0}{AH1}{V}{M}{AY1}{N}{AH0}{S}{F}{AO1}{R}{D}{IH0}{G}{R}{IY1}{Z}{F}{EH1}{R}{AH0}{N}{HH}{AY2}{T}{sp}


In [30]:
print(dur)
print(sigma)

tensor([[ 9.,  6.,  9.,  4.,  9.,  4.,  5.,  8., 11.,  6.,  4.,  6.,  4.,  4.,
          6., 10.,  8.,  4.,  5.,  4.,  4.,  6.,  7.,  5.,  5.,  7.,  3.,  5.,
          8.,  5.,  5.,  5.,  3.,  4., 10., 11.,  3.,  6.,  7., 10.,  4.,  6.,
          8.,  8.,  6.,  5.,  4.,  4.,  6.,  6., 11.,  9.,  8.,  7.,  6.,  3.,
          3., 10., 11., 13., 15.]], device='cuda:0')
tensor([[0.7217, 0.5186, 0.7379, 0.3172, 0.6099, 0.3240, 0.3469, 0.6008, 0.7089,
         0.5009, 0.2231, 0.4686, 0.3242, 0.3111, 0.3626, 0.7170, 0.6366, 0.3392,
         0.3762, 0.3411, 0.2483, 0.4879, 0.5550, 0.3907, 0.2828, 0.5590, 0.2444,
         0.2956, 0.6344, 0.2709, 0.4179, 0.4203, 0.2265, 0.2621, 0.7266, 0.7206,
         0.1919, 0.4796, 0.5259, 0.6601, 0.3295, 0.3841, 0.6307, 0.6418, 0.4913,
         0.3792, 0.3301, 0.2617, 0.4966, 0.4602, 0.6983, 0.6699, 0.6330, 0.4739,
         0.4659, 0.2057, 0.2799, 0.7646, 0.7485, 0.9107, 1.0732]],
       device='cuda:0')


In [31]:
    %matplotlib inline
    align = align.cpu()
    align = align.squeeze(0)
    align = align.T
    fig = plt.figure(figsize=(12, 7))
    plt.imshow(align, aspect='auto', origin='lower', interpolation='none',
        norm=Normalize(vmin=0.0, vmax=1.0))
   
    plt.xlabel('Decoder timestep')
    plt.ylabel('Encoder timestep')
    
    cax = plt.axes([0.9, 0.1, 0.02, 0.8])
    plt.colorbar(cax=cax)

In [32]:
wav_recon = wav_recon.cpu().squeeze(0)
ipd.Audio(wav_recon, rate=hparams.audio.sampling_rate)

In [33]:
    mel_img=mel.squeeze(0)
    mel_img=mel_img.cpu().detach().numpy()
    
    
    fig = plt.figure(figsize=(135,27))
    #cax = plt.axes([0.9, 0.1, 0.02, 0.8])
    #plt.colorbar(format='%+2.0f dB')
    plt.savefig('mel.png')
    plt.imshow(mel_img, aspect = 'auto', origin='lower', interpolation='none')

NameError: name 'mel' is not defined